In [1]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import codecs
%matplotlib inline

In [4]:
#예외처리를 위한 import
from selenium.common.exceptions import NoSuchElementException

In [2]:
working_dir ='본인의 컴퓨터의 NLP_root/description'
NLP_root='본인의 컴퓨터의 NLP_root 설정'

In [5]:
outter_target_count = pd.read_csv(working_dir,'/skipgram/count파일/outter_target/outter_target_count_temp.csv', engine='python',header = None, delimiter=",", quoting=csv.QUOTE_NONE, encoding='utf-8')
# outter_source_count[0][0] # 해당 단어 추출
print(len(outter_target_count[0]))
outter_target_count = outter_target_count[0]

1747


In [7]:
outter_source_count = pd.read_csv(working_dir,'/skipgram/count파일/outter_source/outter_source_count_temp2.csv', engine='python',header = None, delimiter=",", quoting=csv.QUOTE_NONE, encoding='utf-8')
# outter_source_count[0][0] # 해당 단어 추출
print(len(outter_source_count[0]))
outter_source_count_temp = outter_source_count[0]

589


skipgram에 포함되지 않은 {source or target} description 중 오타일 것 같은 것 검색 

In [8]:
outter_target_count=outter_target_count[0]

In [26]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14931")
#크롬 드라이버 경로를 위치시킨다
#크롬 드라이버는 http://chromedriver.chromium.org/downloads 에서 설치가능
driver = webdriver.Chrome(NLP_root,'/chromedriver', options=options)
driver.implicitly_wait(3)
dic_YN=[]
rcm_word=[]
google_YN=[]
google_rcm_word=[]
bigo=[]
for search_item in outter_source_count_temp2:
    url = "https://terms.naver.com"
    driver.get(url)
    driver.find_element_by_id('term_query').send_keys(str(search_item))
    driver.find_element_by_xpath('//*[@id="terms_search_form"]/fieldset/div/input[1]').click()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    notices = soup.select('.relate_desc > em')
    no_result = soup.select('#content > div > p > span > span')
    real_result = soup.select('.subject > strong > a')
    if(len(notices)>0): # 해당 단어의 예상 추천 단어 
        print(search_item,'의 결과는 ',notices[0].text,' 로 예상됨')
        rcm_word = rcm_word + [notices[0].text]
#         print(notices[0].text)
    elif(len(notices)==0):
        rcm_word=rcm_word + ['']
    if(len(no_result)>0): # 사전 검색시 등장 여부 --> NO
        print(search_item,'의 결과는 없음')
        dic_YN = dic_YN+['N']
    elif(len(real_result)>0): # 사전 검색시 등장 여부 --> Yes
        print(search_item,'의 결과가 있음')
        dic_YN= dic_YN +['Y'] 
    driver.implicitly_wait(3)
    
    ######구글 검색
    print('구글 검색!!!!!!!!!!!!!!!!')
    url = "https://www.google.com"
    driver.get(url)
#     ser=driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input').send_keys(str(search_item))
    elem = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input')
    elem.send_keys(str(search_item))
    elem.submit()
#     driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[3]/center/input[1]').click()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    notices = soup.select('#fprsl > em')
    no_result = soup.select('#fprs > span.spell_orig') 
#     real_result = driver.find_element_by_xpath('//*[@id="rso"]/div/div/div[1]/div/div/div[1]/a[1]/h3')
                             
    if(len(notices)>0): # 해당 단어의 예상 추천 단어 
        print(search_item,'의 결과는 ',notices[0].text,' 로 예상됨(구글검색완료)')
        real_result2 = driver.find_element_by_xpath('//*[@id="rso"]/div/div/div[1]/div/div/div[1]/a[1]/h3')
        google_rcm_word = google_rcm_word + [notices[0].text]
        bigo = bigo + [real_result2.text]
#         print(notices[0].text)
    elif(len(notices)==0):
        google_rcm_word=google_rcm_word + ['']
    if(len(no_result)>0): # 구글 검색시 등장 여부 --> NO
        print(search_item,'의 결과는 없음(구글검색완료)')
        google_YN = google_YN+['N']
#         bigo = bigo + ['결과없음']
    
    else: # 구글 검색시 등장 여부 --> Yes    
        try:
            real_result = driver.find_element_by_xpath('//*[@id="rso"]/div/div/div[1]/div/div/div[1]/a[1]/h3')
            print(search_item,'의 결과가 있음(구글검색완료) -->', real_result.text)
            google_YN= google_YN +['Y'] 
            bigo = bigo + [real_result.text]

        except NoSuchElementException:
                print(" 결과없음 ")
                google_YN= google_YN +['N'] 
                bigo = bigo + ['결과없음']
        
    driver.implicitly_wait(3)
    
print(len(dic_YN))
print(len(rcm_word))
print(len(google_YN))
print(len(google_rcm_word))
# a=pd.Series(list(dic_YN))
# b=pd.Series(list(rcm_word))
# d=pd.Series(list(google_YN))
# f=pd.Series(list(google_rcm_word))
e=pd.Series(list(bigo))
# c=pd.concat([outter_source_count,e],axis=1)
dataframe = pd.DataFrame(e)
#한글 깨짐 방지를 위해 excel형식으로 저장. csv로 저장할경우 한글이 깨짐
dataframe.to_excel('C:/Users/admin/Desktop/NLP/description/skipgram/count파일/outter_source/outter_source_count_rcm5.xlsx',sheet_name='sheet1',header=False,index=False)

unspsecified 의 결과는  unspecified  로 예상됨
unspsecified 의 결과가 있음
구글 검색!!!!!!!!!!!!!!!!
unspsecified 의 결과는  unspecified  로 예상됨(구글검색완료)
unspsecified 의 결과는 없음(구글검색완료)
disroder 의 결과는  disorder  로 예상됨
disroder 의 결과가 있음
구글 검색!!!!!!!!!!!!!!!!
disroder 의 결과는  disorder  로 예상됨(구글검색완료)
disroder 의 결과는 없음(구글검색완료)
NTERFERENCE 의 결과는  'interference'  로 예상됨
NTERFERENCE 의 결과가 있음
구글 검색!!!!!!!!!!!!!!!!
NTERFERENCE 의 결과는  INTERFERENCE  로 예상됨(구글검색완료)
NTERFERENCE 의 결과는 없음(구글검색완료)
DAISEUNGMEDICAL 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
DAISEUNGMEDICAL 의 결과는  DAESEUNG MEDICAL  로 예상됨(구글검색완료)
DAISEUNGMEDICAL 의 결과는 없음(구글검색완료)
DERMANETAGBORDER 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
DERMANETAGBORDER 의 결과는  DERMANET AG BORDER  로 예상됨(구글검색완료)
DERMANETAGBORDER 의 결과는 없음(구글검색완료)
NEBLIZER 의 결과는  'nebulizer'  로 예상됨
NEBLIZER 의 결과가 있음
구글 검색!!!!!!!!!!!!!!!!
NEBLIZER 의 결과는  NEBULIZER  로 예상됨(구글검색완료)
NEBLIZER 의 결과는 없음(구글검색완료)
SCIONCARDIO 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
SCIONCARDIO 의 결과는  SCION CARDIO  로 예상됨(구글검색완료)
SCIONCARDIO 의 결과는 없음(구글검색완료)
MULTIF

ACETATEWEAVEGREEN 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
ACETATEWEAVEGREEN 의 결과는  ACETATE WEAVE GREEN  로 예상됨(구글검색완료)
ACETATEWEAVEGREEN 의 결과는 없음(구글검색완료)
DIRECTIONALSKINSTAPLER 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
DIRECTIONALSKINSTAPLER 의 결과는  DIRECTIONAL SKIN STAPLER  로 예상됨(구글검색완료)
DIRECTIONALSKINSTAPLER 의 결과는 없음(구글검색완료)
Renergade 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
Renergade 의 결과는  Renegade  로 예상됨(구글검색완료)
Renergade 의 결과는 없음(구글검색완료)
KICOMPRESSIONE 의 결과는  KICOMPRESSION  로 예상됨
KICOMPRESSIONE 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
KICOMPRESSIONE 의 결과는  KI COMPRESSIONE  로 예상됨(구글검색완료)
KICOMPRESSIONE 의 결과는 없음(구글검색완료)
OHEMEDA 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
OHEMEDA 의 결과는  OHMEDA  로 예상됨(구글검색완료)
OHEMEDA 의 결과는 없음(구글검색완료)
HUBERNEEDLEEXTENSIONSET 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
HUBERNEEDLEEXTENSIONSET 의 결과는  HUBER NEEDLE EXTENSION SET  로 예상됨(구글검색완료)
HUBERNEEDLEEXTENSIONSET 의 결과는 없음(구글검색완료)
RODDINTERSPINOUSDEVICE 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
RODDINTERSPINOUSDEVICE 의 결과는  RODD INTERSPINOUS DEVICE  로 예상됨(구글검색완료)
RODDINTERSPINOU

MULTIFIXIVPLUS 의 결과는  MULTIFIX IV PLUS  로 예상됨(구글검색완료)
MULTIFIXIVPLUS 의 결과는 없음(구글검색완료)
MINIMEDINC 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
MINIMEDINC 의 결과는  MINIMED INC  로 예상됨(구글검색완료)
MINIMEDINC 의 결과는 없음(구글검색완료)
POSITIVENYLON66 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
POSITIVENYLON66 의 결과는  POSITIVE NYLON 66  로 예상됨(구글검색완료)
POSITIVENYLON66 의 결과는 없음(구글검색완료)
Cervial 의 결과가 있음
구글 검색!!!!!!!!!!!!!!!!
Cervial 의 결과는  Cervical  로 예상됨(구글검색완료)
Cervial 의 결과는 없음(구글검색완료)
largd 의 결과는  large  로 예상됨
largd 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
largd 의 결과는  large  로 예상됨(구글검색완료)
largd 의 결과는 없음(구글검색완료)
ADHESIONGEL 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
ADHESIONGEL 의 결과는  ADHESION GEL  로 예상됨(구글검색완료)
ADHESIONGEL 의 결과는 없음(구글검색완료)
LDRMEDICAL 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
LDRMEDICAL 의 결과는  LDR MEDICAL  로 예상됨(구글검색완료)
LDRMEDICAL 의 결과는 없음(구글검색완료)
nonhydatidiform 의 결과가 있음
구글 검색!!!!!!!!!!!!!!!!
nonhydatidiform 의 결과가 있음(구글검색완료) --> ICD-10-CM Code O02.0 - Blighted ovum and nonhydatidiform mole
ANCHORPLUS 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
ANCHORPLUS 의 결과는  

migainosus 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
migainosus 의 결과는  migrainosus  로 예상됨(구글검색완료)
migainosus 의 결과는 없음(구글검색완료)
BSNMEDICALGMBH 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
BSNMEDICALGMBH 의 결과는  BSN MEDICAL GMBH  로 예상됨(구글검색완료)
BSNMEDICALGMBH 의 결과는 없음(구글검색완료)
NICHIBANMEDICALCORP 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
NICHIBANMEDICALCORP 의 결과는  NICHIBAN MEDICAL CORP  로 예상됨(구글검색완료)
NICHIBANMEDICALCORP 의 결과는 없음(구글검색완료)
Menisectomy 의 결과가 있음
구글 검색!!!!!!!!!!!!!!!!
Menisectomy 의 결과는  Meniscectomy  로 예상됨(구글검색완료)
Menisectomy 의 결과는 없음(구글검색완료)
GLICOLIDE 의 결과가 있음
구글 검색!!!!!!!!!!!!!!!!
GLICOLIDE 의 결과는  GLYCOLIDE  로 예상됨(구글검색완료)
GLICOLIDE 의 결과는 없음(구글검색완료)
unespecified 의 결과는  unspecified  로 예상됨
unespecified 의 결과가 있음
구글 검색!!!!!!!!!!!!!!!!
unespecified 의 결과는  unspecified  로 예상됨(구글검색완료)
unespecified 의 결과는 없음(구글검색완료)
ALCHEMILLAVUGARIS 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
ALCHEMILLAVUGARIS 의 결과는  ALCHEMILLA VULGARIS  로 예상됨(구글검색완료)
ALCHEMILLAVUGARIS 의 결과는 없음(구글검색완료)
NEOPHARMCO 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
NEOPHARMCO 의 결과는  NEOPHARM

srorm 의 결과는  storm  로 예상됨(구글검색완료)
srorm 의 결과는 없음(구글검색완료)
NIPPONSIGMAX 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
NIPPONSIGMAX 의 결과는  NIPPON SIGMAX  로 예상됨(구글검색완료)
NIPPONSIGMAX 의 결과는 없음(구글검색완료)
IBUPROPEN 의 결과는  ibuprofen  로 예상됨
IBUPROPEN 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
IBUPROPEN 의 결과는  IBUPROFEN  로 예상됨(구글검색완료)
IBUPROPEN 의 결과는 없음(구글검색완료)
curciate 의 결과는  cruciate  로 예상됨
curciate 의 결과가 있음
구글 검색!!!!!!!!!!!!!!!!
curciate 의 결과는  cruciate  로 예상됨(구글검색완료)
curciate 의 결과는 없음(구글검색완료)
ELASTICPOLYAMIDENET 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
ELASTICPOLYAMIDENET 의 결과는  ELASTIC POLYAMIDE NET  로 예상됨(구글검색완료)
ELASTICPOLYAMIDENET 의 결과는 없음(구글검색완료)
OPTIMASPLINT 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
OPTIMASPLINT 의 결과는  OPTIMA SPLINT  로 예상됨(구글검색완료)
OPTIMASPLINT 의 결과는 없음(구글검색완료)
COLOPLASTA 의 결과는  coloplasty  로 예상됨
COLOPLASTA 의 결과는 없음
구글 검색!!!!!!!!!!!!!!!!
COLOPLASTA 의 결과는  COLOPLAST  로 예상됨(구글검색완료)
COLOPLASTA 의 결과는 없음(구글검색완료)
acromioclavicualr 의 결과는  acromioclavicular  로 예상됨
acromioclavicualr 의 결과가 있음
구글 검색!!!!!!!!!!!!!!!!
acromioclavicua

UnicodeEncodeError: 'cp949' codec can't encode character '\xf3' in position 32: illegal multibyte sequence

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14931")
#크롬 드라이버 경로를 위치시킨다
driver = webdriver.Chrome('C:/Users/admin/Desktop/NLP/chromedriver_win32/chromedriver', options=options)
driver.implicitly_wait(3)
dic_YN=[]
rcm_word=[]
google_YN=[]
google_rcm_word=[]
bigo=[]
for search_item in outter_target_count:
    url = "https://terms.naver.com"
    driver.get(url)
    driver.find_element_by_id('term_query').send_keys(str(search_item))
    driver.find_element_by_xpath('//*[@id="terms_search_form"]/fieldset/div/input[1]').click()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    notices = soup.select('.relate_desc > em')
    no_result = soup.select('#content > div > p > span > span')
    real_result = soup.select('.subject > strong > a')
    if(len(notices)>0): # 해당 단어의 예상 추천 단어 
        print(search_item,'의 결과는 ',notices[0].text,' 로 예상됨')
        rcm_word = rcm_word + [notices[0].text]
#         print(notices[0].text)
    elif(len(notices)==0):
        rcm_word=rcm_word + ['']
    if(len(no_result)>0): # 사전 검색시 등장 여부 --> NO
        print(search_item,'의 결과는 없음')
        dic_YN = dic_YN+['N']
    elif(len(real_result)>0): # 사전 검색시 등장 여부 --> Yes
        print(search_item,'의 결과가 있음')
        dic_YN= dic_YN +['Y'] 
    driver.implicitly_wait(3)
    
    ######구글 검색
    print('구글 검색!!!!!!!!!!!!!!!!')
    url = "https://www.google.com"
    driver.get(url)
#     ser=driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input').send_keys(str(search_item))
    elem = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input')
    elem.send_keys(str(search_item))
    elem.submit()
#     driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[3]/center/input[1]').click()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    notices = soup.select('#fprsl > em')
    no_result = soup.select('#fprs > span.spell_orig') 
#     real_result = driver.find_element_by_xpath('//*[@id="rso"]/div/div/div[1]/div/div/div[1]/a[1]/h3')
                             
    if(len(notices)>0): # 해당 단어의 예상 추천 단어 
        print(search_item,'의 결과는 ',notices[0].text,' 로 예상됨(구글검색완료)')
        google_rcm_word = google_rcm_word + [notices[0].text]
#         print(notices[0].text)
    elif(len(notices)==0):
        google_rcm_word=google_rcm_word + ['']
    if(len(no_result)>0): # 구글 검색시 등장 여부 --> NO
        print(search_item,'의 결과는 없음(구글검색완료)')
        google_YN = google_YN+['N']
    
    else: # 구글 검색시 등장 여부 --> Yes    
        try:
            real_result = driver.find_element_by_xpath('//*[@id="rso"]/div/div/div[1]/div/div/div[1]/a[1]/h3')
            print(search_item,'의 결과가 있음(구글검색완료) -->', real_result.text)
            google_YN= google_YN +['Y'] 
            bigo = bigo + [real_result.text]

        except NoSuchElementException:
                print(" 결과없음 ")
                google_YN= google_YN +['N'] 
                bigo = bigo + ['결과없음']
        
    driver.implicitly_wait(3)
    
print(len(dic_YN))
print(len(rcm_word))
print(len(google_YN))
print(len(google_rcm_word))
a1=pd.Series(list(dic_YN))
b1=pd.Series(list(rcm_word))
d1=pd.Series(list(google_YN))
f1=pd.Series(list(google_rcm_word))
c1=pd.concat([outter_target_count,a1,b1,d1,f1],axis=1)
dataframe = pd.DataFrame(outter_target_count)
dataframe.to_csv('C:/Users/admin/Desktop/NLP/description/skipgram/count파일/outter_target/outter_target_count_rcm.csv',header=False,index=False)

In [ ]:
naver 의학에서 나오지 않은 부분(결과가 없음)을 위키에서 찾아보기

naver 의학에서 나오지 않은 부분(결과가 없음)을 다시 https://medical-dictionary.thefreedictionary.com/allescheriosis
에서 검색해보기 or 수동으로 ㄱㄱ